In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph

In [ ]:
from typing_extensions import TypedDict
from typing import Literal

class TypedDictState(TypedDict):
    name: str
    mood: Literal["happy", "sad"]

In [ ]:
def node_1(state):
    print("---Node 1---")
    return { "name": state["name"] + " is ... "}

def node_2(state):
    print("---Node 2---")
    return { "mood": "happy"}

def node_3(state):
    print("---Node 3---")
    return { "mood": "sad!"}


In [ ]:
import random



def decide_mood(state) -> Literal["node_2", "node_3"]:
    random_number = random.random()
    print(f"random_number: {random_number}")
    if  random_number < 0.5:
        return "node_2"

    return "node_3"

from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

builder = StateGraph(TypedDictState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)

builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))




In [ ]:
graph.invoke({"name" : "Muhammad"})

# DataClass

In [ ]:
from dataclasses import dataclass

@dataclass
class DataClassState:
    name: str
    mood: Literal["happy", "sad"]
    

In [ ]:
def node_1(state):
    print("---Node 1---")
    return { "name": state.name + " is ... "}


builder = StateGraph(DataClassState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)

builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(DataClassState(name="Muhammad", mood="happy"))

## No validation on mood, you can pass anything

In [ ]:
graph.invoke(DataClassState(name="Muhammad", mood="mad"))

# Pydantic

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, validator, ValidationError

class PydanticState(BaseModel):
    name: str
    mood: Literal["happy", "sad"]
    
    @validator('mood')
    def validate_mood(cls, value):
        if value not in ["happy", "sad"]:
            raise ValueError("Each mood must be happy or sad")
        return value
    

try:
    state = PydanticState(name="John Doe", mood="mad")
except ValidationError as e:
    print("Validation Error", e)

In [ ]:
builder = StateGraph(PydanticState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)

builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(PydanticState(name="Muhammad", mood="happy"))

In [ ]:
graph.invoke(PydanticState(name="Muhammad", mood="mad"))